In [1]:
import json
import pandas as pd
from pandas import json_normalize
import numpy as np
from pymongo import MongoClient
import pprint

In [2]:
client = MongoClient("localhost", 27017)
db = client["CitationDB"]
col = db.papers

In [3]:
dbnames = client.list_database_names()
print(dbnames)

['CitationDB', 'admin', 'config', 'local', 'newDB']


In [4]:
db.newDB.count_documents({'year': {"$gte": "2005", "$lte":"2009"}})

0

In [5]:
query ={"year": {"$gte": "2005", "$lte":"2009"}}
#projections = {"id":1, "venue":1, 'n_citation': 1, "year":1}
journal_impact = list(db.col.find(query))


In [6]:
(journal_impact)

[]

In [7]:
df_main = pd.read_pickle("df_main.pkl")
df_refer = pd.read_pickle("df_with_reference.pkl")

In [8]:
title_meta =[]
for index, row in df_main.iterrows():
    if "meta" in row['title']:
        title_meta.append(1)
    else:
        title_meta.append(0)

df_main['title_meta'] = title_meta

In [9]:
title_review =[]
for index, row in df_main.iterrows():
    if "review" in row['title']:
        title_review.append(1)
    else:
        title_review.append(0)

df_main['title_review'] = title_review

In [24]:
with open('journal_impact_1y.json') as json_file:
     impact_1y= json.load(json_file)

In [26]:
impact_1y.pop('') 

621566

In [33]:
impact_1y['']=0

In [38]:
impact_1y_col =[]

for index, row in df_main.iterrows():
    if row['venue'] == "":
        impact_1y_col.append(0)
    else:
        if row['venue'] not in impact_1y.keys():
            impact_1y_col.append(0)
        else:
            for x, y in impact_1y.items():
                if row['venue'] == x:
                    impact_1y_col.append(y)
            

In [40]:
df_main['impact_1y'] = impact_1y_col

In [41]:
df_main['impact_1y'].describe()
#question: should I scale this column?

count     2080.000000
mean      9981.122115
std      14386.152171
min          0.000000
25%        334.500000
50%       3436.000000
75%      14923.750000
max      85811.000000
Name: impact_1y, dtype: float64

In [42]:
#save df3 as df_with_reference
df_main.to_pickle("df_main2.pkl")